In [1]:
import tensorflow as tf
tf.enable_eager_execution()
import keras
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Activation
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping, Callback, ModelCheckpoint
import numpy as np
import pandas as pd
from collections import Counter
import re, os, sys, csv, math, operator

/home/vgf3011/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/vgf3011/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/vgf3011/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/vgf3011/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:5

In [2]:
architecture = '1024x4D-512x3D-256x3D-128x3D-64x2-32x1-1'
activation = 'relu'
dropouts = [0.8, 0.9, 0.7, 0.8]
SEED = 1234567
num_input = 86

In [3]:
def define_model(data, architecture, num_labels=1, activation='relu', dropouts=[]):

        assert '-' in architecture
        archs = architecture.strip().split('-')
        net = data
        pen_layer = net
        prev_layer = net
        prev_num_outputs = None
        prev_block_num_outputs = None
        prev_stub_output = net
        for i in range(len(archs)):
            arch = archs[i]
            if 'x' in arch:
                arch = arch.split('x')
                num_outputs = int(re.findall(r'\d+',arch[0])[0])
                layers = int(re.findall(r'\d+',arch[1])[0])
                j = 0
                aux_layers = re.findall(r'[A-Z]',arch[0])
                for l in range(layers):
                    if aux_layers and aux_layers[0] == 'B':
                        if len(aux_layers)>1 and aux_layers[1]=='A':
                            print('adding fully connected layers with %d outputs followed by batch_norm and act' % num_outputs)

                            net = Dense(num_outputs, 
                                        name='fc' + str(i) + '_' + str(j),
                                        activation=None)(net)
                            net = BatchNormalization(center=True, scale=True, name='fc_bn'+str(i)+'_'+str(j))(net)
                            if activation =='relu': net = Activation('relu')(net)
                        else:
                            print('adding fully connected layers with %d outputs followed by batch_norm' % num_outputs)
                            net = Dense(num_outputs,
                                        name='fc' + str(i) + '_' + str(j),
                                        activation=activation)(net)
                            net = BatchNormalization(center=True, scale=True,
                                             name='fc_bn' + str(i) + '_' + str(j))(net)

                    else:
                        print('adding fully connected layers with %d outputs' % num_outputs)

                        net = Dense(num_outputs,
                                    name='fc' + str(i) + '_' + str(j), 
                                    activation=activation)(net)

                    if 'R' in aux_layers:
                        if prev_num_outputs and prev_num_outputs==num_outputs:
                            print('adding residual, both sizes are same')

                            net = net+prev_layer
                        else:
                            print('adding residual with fc as the size are different')
                            net = net + Dense(num_outputs,
                                                name='fc' + str(i) + '_' +'dim_'+ str(j),
                                                activation=None)(prev_layer)
                    prev_num_outputs = num_outputs
                    j += 1
                    prev_layer = net
                aux_layers_sub = re.findall(r'[A-Z]', arch[1])
                if 'R' in aux_layers_sub:
                    if prev_block_num_outputs and prev_block_num_outputs == num_outputs:
                        print('adding residual to stub, both sizes are same')
                        net = net + prev_stub_output
                    else:
                        print('adding residual to stub with fc as the size are different')
                        net = net + Dense(num_outputs,
                                         name='fc' + str(i) + '_' + 'stub_dim_' + str(j),
                                         activation=None)(prev_stub_output)

                if 'D' in aux_layers_sub and (num_labels == 1) and len(dropouts) > i:
                    print('adding dropout', dropouts[i])
                    net = Dropout(1.-dropouts[i], seed=SEED)(net, training=False)
                prev_stub_output = net
                prev_block_num_outputs = num_outputs
                prev_layer = net

            else:
                if 'R' in arch:
                    act_fun = 'relu'
                    print('using ReLU at last layer')  
                else:
                    act_fun = None
                pen_layer = net
                print('adding final layer with ' + str(num_labels) + ' output')
                net = Dense(num_labels, name='fc' + str(i),
                            activation=act_fun)(net)

        return net

In [4]:
inputs = Input(shape=(num_input,), name='elemental_fractions')
outputs = define_model(inputs, architecture, dropouts=dropouts)
model = Model(inputs=inputs, outputs=outputs, name= 'ElemNet')
model.summary(print_fn=lambda x: print(x))

model.load_weights("model/model_elemnet_nature_wojarvis_new_training-false_tf2_1.h5")
#model.load_weights("model/model_elemnet_nature_new_training-false_tf2_1.h5")
#model.load_weights("model/model_elemnet_nature_noexp_trf_tf2_1.h5")
#model.load_weights("model/model_elemnet_nature_noexpbg_trf_tf2_1.h5")

#model.load_weights("model/model_elemnet_nature_wooqmdcs_new_trf_tf2_1.h5")

print("Loaded model from disk")

adam = optimizers.Adam(lr=0.0001)
model.compile(loss=tf.keras.losses.mean_absolute_error, optimizer=adam, metrics=['mean_absolute_error'])

adding fully connected layers with 1024 outputs
adding fully connected layers with 1024 outputs
adding fully connected layers with 1024 outputs
adding fully connected layers with 1024 outputs
adding dropout 0.8
adding fully connected layers with 512 outputs
adding fully connected layers with 512 outputs
adding fully connected layers with 512 outputs
adding dropout 0.9
adding fully connected layers with 256 outputs
adding fully connected layers with 256 outputs
adding fully connected layers with 256 outputs
adding dropout 0.7
adding fully connected layers with 128 outputs
adding fully connected layers with 128 outputs
adding fully connected layers with 128 outputs
adding dropout 0.8
adding fully connected layers with 64 outputs
adding fully connected layers with 64 outputs
adding fully connected layers with 32 outputs
adding final layer with 1 output
Model: "ElemNet"
_________________________________________________________________
Layer (type)                 Output Shape              

In [5]:
#Contains 86 elements (Without Noble elements as it does not forms compounds in normal condition)
elements = ['H','Li','Be', 'B', 'C', 'N', 'O', 'F', 'Na', 'Mg', 'Al', 'Si', 'P', 'S', 'Cl',
            'K', 'Ca', 'Sc', 'Ti', 'V', 'Cr', 'Mn', 'Fe','Co', 'Ni', 'Cu', 'Zn', 'Ga', 'Ge',
            'As', 'Se', 'Br', 'Kr', 'Rb', 'Sr', 'Y', 'Zr', 'Nb', 'Mo', 'Tc', 'Ru', 'Rh', 'Pd',
            'Ag', 'Cd', 'In', 'Sn', 'Sb', 'Te', 'I', 'Xe', 'Cs', 'Ba', 'La', 'Ce', 'Pr', 'Nd',
            'Pm', 'Sm', 'Eu', 'Gd', 'Tb', 'Dy', 'Ho', 'Er','Tm', 'Yb', 'Lu', 'Hf', 'Ta', 'W', 
            'Re', 'Os', 'Ir', 'Pt', 'Au', 'Hg', 'Tl', 'Pb', 'Bi', 'Ac', 'Th', 'Pa', 'U', 'Np', 'Pu' ]

elements_all = ['H', 'He', 'Li', 'Be', 'B', 'C', 'N', 'O', 'F', 'Ne', 'Na', 'Mg', 'Al', 'Si', 
                'P', 'S', 'Cl', 'Ar', 'K', 'Ca', 'Sc', 'Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni',
                'Cu', 'Zn', 'Ga', 'Ge', 'As', 'Se', 'Br', 'Kr', 'Rb', 'Sr', 'Y', 'Zr', 'Nb', 
                'Mo', 'Tc', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd', 'In', 'Sn', 'Sb', 'Te', 'I', 'Xe',
                'Cs', 'Ba', 'La', 'Ce', 'Pr', 'Nd', 'Pm', 'Sm', 'Eu', 'Gd', 'Tb', 'Dy', 'Ho',
                'Er', 'Tm', 'Yb', 'Lu', 'Hf', 'Ta', 'W', 'Re', 'Os', 'Ir', 'Pt', 'Au', 'Hg',
                'Tl', 'Pb', 'Bi', 'Po', 'At', 'Rn', 'Fr', 'Ra', 'Ac', 'Th', 'Pa', 'U', 'Np',
                'Pu', 'Am', 'Cm', 'Bk', 'Cf', 'Es', 'Fm', 'Md', 'No', 'Lr', 'Rf', 'Db', 'Sg',
                'Bh', 'Hs', 'Mt', 'Ds', 'Rg', 'Cn']

# Regex to Choose from Element Name, Number and Either of the Brackets
token = re.compile('[A-Z][a-z]?|\d+|[()]')

# Create a dictionary with the Name of the Element as Key and No. of elements as Value
def count_elements(formula):
    tokens = token.findall(str(formula))
    stack = [[]]
    for t in tokens:
        if t.isalpha():
            last = [t]
            stack[-1].append(t)
        elif t.isdigit():
             stack[-1].extend(last*(int(t)-1))
        elif t == '(':
            stack.append([])
        elif t == ')':
            last = stack.pop()
            stack[-1].extend(last)   
    return dict(Counter(stack[-1]))

#Normalize the Value of the Dictionary
def normalize_elements(dictionary):
    dic_val = sum(dictionary.values()) 
    if dic_val == 0:
        factor = 0
    else:    
        factor=1.0/ dic_val  
        
    for k in dictionary:
        dictionary[k] = dictionary[k]*factor
    return dictionary

def Diff(li1, li2): 
    return (list(set(li1) - set(li2))) 

print(Diff(elements_all, elements)) 

def elemental_fraction(dataframe):
    print('The loaded dataset has %d entries'%len(dataframe['pretty_comp']))

    #data = mp_data[mp_data['composition'].str.contains('(H|Li|Be|B|C|N|O|F|Na|Mg|Al|Si|P|S|Cl|K|Ca|Sc|Ti|V|Cr|Mn|Fe|Co|Ni|Cu|Zn|Ga|Ge|As|Se|Br|Kr|Rb|Sr|Y|Zr|Nb|Mo|Tc|Ru|Rh|Pd|Ag|Cd|In|Sn|Sb|Te|I|Xe|Cs|Ba|La|Ce|Pr|Nd|Pm|Sm|Eu|Gd|Tb|Dy|Ho|Er|Tm|Yb|Lu|Hf|Ta|W|Re|Os|Ir|Pt|Au|Hg|Tl|Pb|Bi|Ac|Th|Pa|U|Np|Pu)[0-9]+', case=True)]
    #data = dataframe[~dataframe['pretty_comp'].str.contains('Bk|Md|Ds|Sg|Ar|No|At|Db|He|Po|Fr|Cm|Cn|Rn|Mt|Fm|Cf|Hs|Ra|Es|Bh|Rf|Lr|Rg|Ne|Am')]
    compounds = dataframe['pretty_comp']

    print('The reduced dataset has %d entries'%len(compounds))
    
    compounds = [count_elements(x) for x in compounds]
    compounds = [normalize_elements(x) for x in compounds]

    in_elements = np.zeros(shape=(len(compounds), len(elements)))
    comp_no = 0

    for compound in compounds:
        keys = compound.keys()
        for key in keys:
            in_elements[comp_no][elements.index(key)] = compound[key]
        comp_no+=1  
    
    data = in_elements
    
    return data


phy_prop = ['0-norm', '2-norm', '3-norm', '5-norm', '7-norm', '10-norm', 'MagpieData minimum Number', 'MagpieData maximum Number', 'MagpieData range Number', 'MagpieData mean Number', 'MagpieData avg_dev Number', 'MagpieData mode Number', 'MagpieData minimum MendeleevNumber', 'MagpieData maximum MendeleevNumber', 'MagpieData range MendeleevNumber', 'MagpieData mean MendeleevNumber', 'MagpieData avg_dev MendeleevNumber', 'MagpieData mode MendeleevNumber', 'MagpieData minimum AtomicWeight', 'MagpieData maximum AtomicWeight', 'MagpieData range AtomicWeight', 'MagpieData mean AtomicWeight', 'MagpieData avg_dev AtomicWeight', 'MagpieData mode AtomicWeight', 'MagpieData minimum MeltingT', 'MagpieData maximum MeltingT', 'MagpieData range MeltingT', 'MagpieData mean MeltingT', 'MagpieData avg_dev MeltingT', 'MagpieData mode MeltingT', 'MagpieData minimum Column', 'MagpieData maximum Column', 'MagpieData range Column', 'MagpieData mean Column', 'MagpieData avg_dev Column', 'MagpieData mode Column', 'MagpieData minimum Row', 'MagpieData maximum Row', 'MagpieData range Row', 'MagpieData mean Row', 'MagpieData avg_dev Row', 'MagpieData mode Row', 'MagpieData minimum CovalentRadius', 'MagpieData maximum CovalentRadius', 'MagpieData range CovalentRadius', 'MagpieData mean CovalentRadius', 'MagpieData avg_dev CovalentRadius', 'MagpieData mode CovalentRadius', 'MagpieData minimum Electronegativity', 'MagpieData maximum Electronegativity', 'MagpieData range Electronegativity', 'MagpieData mean Electronegativity', 'MagpieData avg_dev Electronegativity', 'MagpieData mode Electronegativity', 'MagpieData minimum NsValence', 'MagpieData maximum NsValence', 'MagpieData range NsValence', 'MagpieData mean NsValence', 'MagpieData avg_dev NsValence', 'MagpieData mode NsValence', 'MagpieData minimum NpValence', 'MagpieData maximum NpValence', 'MagpieData range NpValence', 'MagpieData mean NpValence', 'MagpieData avg_dev NpValence', 'MagpieData mode NpValence', 'MagpieData minimum NdValence', 'MagpieData maximum NdValence', 'MagpieData range NdValence', 'MagpieData mean NdValence', 'MagpieData avg_dev NdValence', 'MagpieData mode NdValence', 'MagpieData minimum NfValence', 'MagpieData maximum NfValence', 'MagpieData range NfValence', 'MagpieData mean NfValence', 'MagpieData avg_dev NfValence', 'MagpieData mode NfValence', 'MagpieData minimum NValence', 'MagpieData maximum NValence', 'MagpieData range NValence', 'MagpieData mean NValence', 'MagpieData avg_dev NValence', 'MagpieData mode NValence', 'MagpieData minimum NsUnfilled', 'MagpieData maximum NsUnfilled', 'MagpieData range NsUnfilled', 'MagpieData mean NsUnfilled', 'MagpieData avg_dev NsUnfilled', 'MagpieData mode NsUnfilled', 'MagpieData minimum NpUnfilled', 'MagpieData maximum NpUnfilled', 'MagpieData range NpUnfilled', 'MagpieData mean NpUnfilled', 'MagpieData avg_dev NpUnfilled', 'MagpieData mode NpUnfilled', 'MagpieData minimum NdUnfilled', 'MagpieData maximum NdUnfilled', 'MagpieData range NdUnfilled', 'MagpieData mean NdUnfilled', 'MagpieData avg_dev NdUnfilled', 'MagpieData mode NdUnfilled', 'MagpieData minimum NfUnfilled', 'MagpieData maximum NfUnfilled', 'MagpieData range NfUnfilled', 'MagpieData mean NfUnfilled', 'MagpieData avg_dev NfUnfilled', 'MagpieData mode NfUnfilled', 'MagpieData minimum NUnfilled', 'MagpieData maximum NUnfilled', 'MagpieData range NUnfilled', 'MagpieData mean NUnfilled', 'MagpieData avg_dev NUnfilled', 'MagpieData mode NUnfilled', 'MagpieData minimum GSvolume_pa', 'MagpieData maximum GSvolume_pa', 'MagpieData range GSvolume_pa', 'MagpieData mean GSvolume_pa', 'MagpieData avg_dev GSvolume_pa', 'MagpieData mode GSvolume_pa', 'MagpieData minimum GSbandgap', 'MagpieData maximum GSbandgap', 'MagpieData range GSbandgap', 'MagpieData mean GSbandgap', 'MagpieData avg_dev GSbandgap', 'MagpieData mode GSbandgap', 'MagpieData minimum GSmagmom', 'MagpieData maximum GSmagmom', 'MagpieData range GSmagmom', 'MagpieData mean GSmagmom', 'MagpieData avg_dev GSmagmom', 'MagpieData mode GSmagmom', 'MagpieData minimum SpaceGroupNumber', 'MagpieData maximum SpaceGroupNumber', 'MagpieData range SpaceGroupNumber', 'MagpieData mean SpaceGroupNumber', 'MagpieData avg_dev SpaceGroupNumber', 'MagpieData mode SpaceGroupNumber', 'avg s valence electrons', 'avg p valence electrons', 'avg d valence electrons', 'avg f valence electrons', 'compound possible', 'max ionic char', 'avg ionic char']

all_atts = ['mean_EffectiveCoordination', 'var_EffectiveCoordination', 'min_EffectiveCoordination', 'max_EffectiveCoordination', 'var_MeanBondLength', 'min_MeanBondLength', 'max_MeanBondLength', 'mean_BondLengthVariation', 'var_BondLengthVariation', 'min_BondLengthVariation', 'max_BondLengthVariation', 'var_CellVolume', 'mean_WCMagnitude_Shell1', 'mean_WCMagnitude_Shell2', 'mean_WCMagnitude_Shell3', 'MaxPackingEfficiency', 'mean_NeighDiff_shell1_Number', 'var_NeighDiff_shell1_Number', 'min_NeighDiff_shell1_Number', 'max_NeighDiff_shell1_Number', 'range_NeighDiff_shell1_Number', 'mean_NeighDiff_shell1_MendeleevNumber', 'var_NeighDiff_shell1_MendeleevNumber', 'min_NeighDiff_shell1_MendeleevNumber', 'max_NeighDiff_shell1_MendeleevNumber', 'range_NeighDiff_shell1_MendeleevNumber', 'mean_NeighDiff_shell1_AtomicWeight', 'var_NeighDiff_shell1_AtomicWeight', 'min_NeighDiff_shell1_AtomicWeight', 'max_NeighDiff_shell1_AtomicWeight', 'range_NeighDiff_shell1_AtomicWeight', 'mean_NeighDiff_shell1_MeltingT', 'var_NeighDiff_shell1_MeltingT', 'min_NeighDiff_shell1_MeltingT', 'max_NeighDiff_shell1_MeltingT', 'range_NeighDiff_shell1_MeltingT', 'mean_NeighDiff_shell1_Column', 'var_NeighDiff_shell1_Column', 'min_NeighDiff_shell1_Column', 'max_NeighDiff_shell1_Column', 'range_NeighDiff_shell1_Column', 'mean_NeighDiff_shell1_Row', 'var_NeighDiff_shell1_Row', 'min_NeighDiff_shell1_Row', 'max_NeighDiff_shell1_Row', 'range_NeighDiff_shell1_Row', 'mean_NeighDiff_shell1_CovalentRadius', 'var_NeighDiff_shell1_CovalentRadius', 'min_NeighDiff_shell1_CovalentRadius', 'max_NeighDiff_shell1_CovalentRadius', 'range_NeighDiff_shell1_CovalentRadius', 'mean_NeighDiff_shell1_Electronegativity', 'var_NeighDiff_shell1_Electronegativity', 'min_NeighDiff_shell1_Electronegativity', 'max_NeighDiff_shell1_Electronegativity', 'range_NeighDiff_shell1_Electronegativity', 'mean_NeighDiff_shell1_NsValence', 'var_NeighDiff_shell1_NsValence', 'min_NeighDiff_shell1_NsValence', 'max_NeighDiff_shell1_NsValence', 'range_NeighDiff_shell1_NsValence', 'mean_NeighDiff_shell1_NpValence', 'var_NeighDiff_shell1_NpValence', 'min_NeighDiff_shell1_NpValence', 'max_NeighDiff_shell1_NpValence', 'range_NeighDiff_shell1_NpValence', 'mean_NeighDiff_shell1_NdValence', 'var_NeighDiff_shell1_NdValence', 'min_NeighDiff_shell1_NdValence', 'max_NeighDiff_shell1_NdValence', 'range_NeighDiff_shell1_NdValence', 'mean_NeighDiff_shell1_NfValence', 'var_NeighDiff_shell1_NfValence', 'min_NeighDiff_shell1_NfValence', 'max_NeighDiff_shell1_NfValence', 'range_NeighDiff_shell1_NfValence', 'mean_NeighDiff_shell1_NValance', 'var_NeighDiff_shell1_NValance', 'min_NeighDiff_shell1_NValance', 'max_NeighDiff_shell1_NValance', 'range_NeighDiff_shell1_NValance', 'mean_NeighDiff_shell1_NsUnfilled', 'var_NeighDiff_shell1_NsUnfilled', 'min_NeighDiff_shell1_NsUnfilled', 'max_NeighDiff_shell1_NsUnfilled', 'range_NeighDiff_shell1_NsUnfilled', 'mean_NeighDiff_shell1_NpUnfilled', 'var_NeighDiff_shell1_NpUnfilled', 'min_NeighDiff_shell1_NpUnfilled', 'max_NeighDiff_shell1_NpUnfilled', 'range_NeighDiff_shell1_NpUnfilled', 'mean_NeighDiff_shell1_NdUnfilled', 'var_NeighDiff_shell1_NdUnfilled', 'min_NeighDiff_shell1_NdUnfilled', 'max_NeighDiff_shell1_NdUnfilled', 'range_NeighDiff_shell1_NdUnfilled', 'mean_NeighDiff_shell1_NfUnfilled', 'var_NeighDiff_shell1_NfUnfilled', 'min_NeighDiff_shell1_NfUnfilled', 'max_NeighDiff_shell1_NfUnfilled', 'range_NeighDiff_shell1_NfUnfilled', 'mean_NeighDiff_shell1_NUnfilled', 'var_NeighDiff_shell1_NUnfilled', 'min_NeighDiff_shell1_NUnfilled', 'max_NeighDiff_shell1_NUnfilled', 'range_NeighDiff_shell1_NUnfilled', 'mean_NeighDiff_shell1_GSvolume_pa', 'var_NeighDiff_shell1_GSvolume_pa', 'min_NeighDiff_shell1_GSvolume_pa', 'max_NeighDiff_shell1_GSvolume_pa', 'range_NeighDiff_shell1_GSvolume_pa', 'mean_NeighDiff_shell1_GSbandgap', 'var_NeighDiff_shell1_GSbandgap', 'min_NeighDiff_shell1_GSbandgap', 'max_NeighDiff_shell1_GSbandgap', 'range_NeighDiff_shell1_GSbandgap', 'mean_NeighDiff_shell1_GSmagmom', 'var_NeighDiff_shell1_GSmagmom', 'min_NeighDiff_shell1_GSmagmom', 'max_NeighDiff_shell1_GSmagmom', 'range_NeighDiff_shell1_GSmagmom', 'mean_NeighDiff_shell1_SpaceGroupNumber', 'var_NeighDiff_shell1_SpaceGroupNumber', 'min_NeighDiff_shell1_SpaceGroupNumber', 'max_NeighDiff_shell1_SpaceGroupNumber', 'range_NeighDiff_shell1_SpaceGroupNumber', 'NComp', 'Comp_L2Norm', 'Comp_L3Norm', 'Comp_L5Norm', 'Comp_L7Norm', 'Comp_L10Norm', 'mean_Number', 'maxdiff_Number', 'dev_Number', 'max_Number', 'min_Number', 'most_Number', 'mean_MendeleevNumber', 'maxdiff_MendeleevNumber', 'dev_MendeleevNumber', 'max_MendeleevNumber', 'min_MendeleevNumber', 'most_MendeleevNumber', 'mean_AtomicWeight', 'maxdiff_AtomicWeight', 'dev_AtomicWeight', 'max_AtomicWeight', 'min_AtomicWeight', 'most_AtomicWeight', 'mean_MeltingT', 'maxdiff_MeltingT', 'dev_MeltingT', 'max_MeltingT', 'min_MeltingT', 'most_MeltingT', 'mean_Column', 'maxdiff_Column', 'dev_Column', 'max_Column', 'min_Column', 'most_Column', 'mean_Row', 'maxdiff_Row', 'dev_Row', 'max_Row', 'min_Row', 'most_Row', 'mean_CovalentRadius', 'maxdiff_CovalentRadius', 'dev_CovalentRadius', 'max_CovalentRadius', 'min_CovalentRadius', 'most_CovalentRadius', 'mean_Electronegativity', 'maxdiff_Electronegativity', 'dev_Electronegativity', 'max_Electronegativity', 'min_Electronegativity', 'most_Electronegativity', 'mean_NsValence', 'maxdiff_NsValence', 'dev_NsValence', 'max_NsValence', 'min_NsValence', 'most_NsValence', 'mean_NpValence', 'maxdiff_NpValence', 'dev_NpValence', 'max_NpValence', 'min_NpValence', 'most_NpValence', 'mean_NdValence', 'maxdiff_NdValence', 'dev_NdValence', 'max_NdValence', 'min_NdValence', 'most_NdValence', 'mean_NfValence', 'maxdiff_NfValence', 'dev_NfValence', 'max_NfValence', 'min_NfValence', 'most_NfValence', 'mean_NValance', 'maxdiff_NValance', 'dev_NValance', 'max_NValance', 'min_NValance', 'most_NValance', 'mean_NsUnfilled', 'maxdiff_NsUnfilled', 'dev_NsUnfilled', 'max_NsUnfilled', 'min_NsUnfilled', 'most_NsUnfilled', 'mean_NpUnfilled', 'maxdiff_NpUnfilled', 'dev_NpUnfilled', 'max_NpUnfilled', 'min_NpUnfilled', 'most_NpUnfilled', 'mean_NdUnfilled', 'maxdiff_NdUnfilled', 'dev_NdUnfilled', 'max_NdUnfilled', 'min_NdUnfilled', 'most_NdUnfilled', 'mean_NfUnfilled', 'maxdiff_NfUnfilled', 'dev_NfUnfilled', 'max_NfUnfilled', 'min_NfUnfilled', 'most_NfUnfilled', 'mean_NUnfilled', 'maxdiff_NUnfilled', 'dev_NUnfilled', 'max_NUnfilled', 'min_NUnfilled', 'most_NUnfilled', 'mean_GSvolume_pa', 'maxdiff_GSvolume_pa', 'dev_GSvolume_pa', 'max_GSvolume_pa', 'min_GSvolume_pa', 'most_GSvolume_pa', 'mean_GSbandgap', 'maxdiff_GSbandgap', 'dev_GSbandgap', 'max_GSbandgap', 'min_GSbandgap', 'most_GSbandgap', 'mean_GSmagmom', 'maxdiff_GSmagmom', 'dev_GSmagmom', 'max_GSmagmom', 'min_GSmagmom', 'most_GSmagmom', 'mean_SpaceGroupNumber', 'maxdiff_SpaceGroupNumber', 'dev_SpaceGroupNumber', 'max_SpaceGroupNumber', 'min_SpaceGroupNumber', 'most_SpaceGroupNumber', 'frac_sValence', 'frac_pValence', 'frac_dValence', 'frac_fValence', 'CanFormIonic', 'MaxIonicChar', 'MeanIonicChar']

['Cn', 'Cf', 'Fm', 'Fr', 'Po', 'Md', 'Ar', 'Rf', 'Ds', 'Hs', 'Rn', 'Db', 'Lr', 'No', 'Am', 'At', 'Ra', 'Bk', 'Sg', 'Es', 'He', 'Bh', 'Mt', 'Cm', 'Rg', 'Ne']


In [27]:
pred = pd.read_csv(r'../../ElemNet_vishu/matminer/jarvis_exfoliation_energy_new.csv') 
#pred = pd.read_csv(r'../data/holdout10/experimental_training_set.csv') 
#pred = pd.read_csv(r'../../ElemNet_vishu/matminer/expt_gap_c.csv') 
#pred = pd.read_csv(r'../../ElemNet_vishu/matminer/oqmd_crystalsystem.csv')
#oqmd = oqmd[~oqmd['formulae'].str.contains('Bk|Md|Ds|Sg|Ar|No|At|Db|He|Po|Fr|Cm|Cn|Rn|Mt|Fm|Cf|Hs|Ra|Es|Bh|Rf|Lr|Rg|Ne|Am', na=False)]

print(pred.shape)    

pred_prop = pred[elements]

#train_phy['compound possible'] = train_phy['compound possible'].astype(float)

new_x_pred = pred_prop.values

new_x_pred = np.asarray(new_x_pred, dtype=np.float)

y_pred = pred.pop('exfoliation_energy').to_frame()

new_y_pred = np.array(y_pred)

new_y_pred.shape = (len(new_y_pred),)

#pred["crystal_system"]= pred["crystal_system"].replace("monoclinic", 1)
#pred["crystal_system"]= pred["crystal_system"].replace("triclinic", 2)
#pred["crystal_system"]= pred["crystal_system"].replace("cubic", 3)
#pred["crystal_system"]= pred["crystal_system"].replace("trigonal", 4)
#pred["crystal_system"]= pred["crystal_system"].replace("tetragonal", 5)
#pred["crystal_system"]= pred["crystal_system"].replace("orthorhombic", 6)
#pred["crystal_system"]= pred["crystal_system"].replace("hexagonal", 7)

(557, 273)


In [28]:
extractor = tf.keras.models.Model(inputs=model.inputs, outputs=[layer.output for layer in model.layers])
features = extractor(new_x_pred)
len(features)
#composition = pred.pop('composition').to_frame()
composition = pred.pop('pretty_comp').to_frame()

In [29]:
for i in range(len(features)):
    globals()["f" + str(i)] = features[i].numpy()
    globals()["df" + str(i)] = pd.DataFrame(data=globals()["f" + str(i)])
    globals()["ndf" + str(i)] = pd.concat([composition,globals()["df" + str(i)],y_pred], axis=1)
    path = '../data/jarvistfowoj_exfoliationenergy/jarvis_exfoliationenergy_f{}.csv'.format(i)
    globals()["ndf" + str(i)].to_csv(path, index=False)

In [50]:
#inlayer = [1024, 1024, 1024, 1024, 512, 512, 512, 256, 256, 256, 128, 128, 128, 64, 64, 32]
#for i in range(len(features)):
#    globals()["f" + str(i)] = features[i].numpy()
#    globals()["df" + str(i)] = pd.DataFrame(data=globals()["f" + str(i)])
#    globals()["ndf" + str(i)] = pd.concat([composition,globals()["df" + str(i)],y_pred], axis=1)
#    path = "../data/holdout10/oqmdsctrf/exp10_layer{}_{}/exp10_test_set.csv".format(i+1, inlayer[i])
#    globals()["ndf" + str(i)].to_csv(path, index=False)